In [9]:
import tensorflow as tf
import cv2
import os
from glob import glob
import numpy as np

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

alpha = 1

#load trained model / 학습된 이미지 불러오기
Generator = tf.keras.models.load_model("Generator.h5")
Generator.set_weights(np.array(Generator.weights) * alpha + np.array(tf.keras.models.load_model('./Generator_PSNR.h5').weights) * (1 - alpha))
Generator.trainable = False

C:\Users\abc\AppData\Local\Temp\ipykernel_18648\3443236957.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Generator.set_weights(np.array(Generator.weights) * alpha + np.array(tf.keras.models.load_model('./Generator_PSNR.h5').weights) * (1 - alpha))


In [10]:
CFG = {"VIDEO_DIR": './video/',
       "SAVE_DIR": './video/'}

In [19]:
def vid_sr(filepath):
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)

    #불러온 비디오 파일의 정보 출력
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)

    count=0
    frame_dirs = []
    if not os.path.isdir(filepath[:-4]):
        os.mkdir(filepath[:-4])
    while True:
        ret, image = video.read()
        if not ret: break
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)/255
        save_dir = filepath[:-4] + "/frame%s.png" % str(count).zfill(8)
        count += 1
        frame_dirs.append(save_dir)
        if len(frame_dirs) >= 100: break
        if os.path.isfile(save_dir): continue

        image = Generator.predict(np.expand_dims(image, 0))
        image[image >= 1] = 1
        image[image <= -1] = -1
        image = np.array(image[0] * 127.5 + 127.5)
        image = np.array(image, dtype=np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_dir, image)
    video.release()

    frame_array = []
    for dir in frame_dirs: 
        img = cv2.imread(dir)
        print(dir)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
        
    out = cv2.VideoWriter(filepath[:-4] + "_SR.mp4",cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [20]:
def vid_lr(filepath, ratio=2):
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)

    #불러온 비디오 파일의 정보 출력
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)

    count=0
    frame_dirs = []
    if not os.path.isdir(filepath[:-4]):
        os.mkdir(filepath[:-4])
    while True:
        ret, image = video.read()
        if not ret: break
        save_dir = filepath[:-4] + "/frame%s.png" % str(count).zfill(8)
        count += 1
        frame_dirs.append(save_dir)
        if os.path.isfile(save_dir): continue
        image = cv2.resize(image, dsize=(width//ratio, height//ratio), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(save_dir, image)

    video.release()

    frame_array = []
    print(frame_dirs)
    for dir in frame_dirs: 
        img = cv2.imread(dir)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
        
    print(frame_array)
    out = cv2.VideoWriter(filepath[:-4] + "_SR.mp4",cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [21]:
for dir in glob(CFG['VIDEO_DIR'] + "/*"):
    if os.path.isfile(dir):
        vid_sr(dir)

length : 408
width : 640
height : 360
fps : 23.976023976023978
./video\reeds-152740/frame00000000.png
./video\reeds-152740/frame00000001.png
./video\reeds-152740/frame00000002.png
./video\reeds-152740/frame00000003.png
./video\reeds-152740/frame00000004.png
./video\reeds-152740/frame00000005.png
./video\reeds-152740/frame00000006.png
./video\reeds-152740/frame00000007.png
./video\reeds-152740/frame00000008.png
./video\reeds-152740/frame00000009.png
./video\reeds-152740/frame00000010.png
./video\reeds-152740/frame00000011.png
./video\reeds-152740/frame00000012.png
./video\reeds-152740/frame00000013.png
./video\reeds-152740/frame00000014.png
./video\reeds-152740/frame00000015.png
./video\reeds-152740/frame00000016.png
./video\reeds-152740/frame00000017.png
./video\reeds-152740/frame00000018.png
./video\reeds-152740/frame00000019.png
./video\reeds-152740/frame00000020.png
./video\reeds-152740/frame00000021.png
./video\reeds-152740/frame00000022.png
./video\reeds-152740/frame00000023.png
.